<a href="https://colab.research.google.com/github/stephenkiilu/NLP_Week1/blob/main/Stephen_Kiilu_n_gram_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 style="font-family:verdana;font-size:300%;text-align:center;background-color:#f2f2f2;color:#0d0d0d">AMMI NLP - Review sessions</h1>

<h1 style="font-family:verdana;font-size:180%;text-align:Center;color:#993333"> Lab 3: n-gram models </h1>

**Big thanks to Amr Khalifa who improved this lab and made it to a Jupyter Notebook!**

In [23]:
import io, sys, math, re
from collections import defaultdict
import numpy as np

In [24]:
# data_loader
def load_data(filename):
    '''
    parameters:
    filename (string): datafile
    
    Returns:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    '''
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    vocab = defaultdict(lambda:0)
    for line in fin:
        sentence = line.split()
        data.append(sentence)
        for word in sentence:
            vocab[word] += 1
    return data, vocab

In [3]:
# file=open('/content/train1.txt',encoding='utf-8')
# files=file.readlines()
# files[0:5]

In [11]:
# dat=[]
# v = defaultdict(lambda:0)
# for sent in files:
#   sentences= sent.split()
#   dat.append(sentences)
#   for w in sentences:
#    v[w]+=1
# print(v)


# train_data, vocab = load_data('/content/train1.txt')


In [25]:

print("load training set..")
print("\n")
train_data, vocab = load_data('/content/train2.txt')
print(train_data[0])
print("\n")
print("how :",vocab['how'])
print("load validation set")
valid_data, _ = load_data("/content/valid2.txt")


load training set..


['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>']


how : 3195
load validation set


In [27]:
valid_data[0]

['<s>',
 'the',
 'wild',
 'horse',
 'was',
 'finally',
 'broken',
 'by',
 'the',
 'patient',
 '<unk>',
 '.',
 '</s>']

In [31]:
def remove_rare_words(data, vocab, mincount = 1):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    vocab (dictionary): {word: no of times it appears in the text}
    mincount(int): the minimum count 
    
    Returns: 
    data_with_unk(list of lists): data after replacing rare words with <unk> token
    '''
    # replace words in data that are not in the vocab 
    # or have a count that is below mincount
    data_with_unk = []

    for sen in data:
      s=[]
      for w in sen:
        if vocab[w]<=mincount or w not in vocab:
          s.append('<UKN>')
        else:
          s.append(w)
      data_with_unk.append(s)

    ## FILL CODE
    
    return data_with_unk

In [32]:
print("remove rare words")
train_data = remove_rare_words(train_data, vocab, mincount = 1)
valid_data = remove_rare_words(valid_data, vocab, mincount = 1)
#train_data
print(train_data[0:5])


remove rare words
[['<s>', 'i', 'liked', 'your', 'idea', 'and', 'adopted', 'it', '.', '</s>'], ['<s>', 'you', 'are', 'wrong', ',', 'however', '.', '</s>'], ['<s>', 'how', 'soon', 'will', 'this', 'laundry', 'be', 'ready', '?', '</s>'], ['<s>', 'it', 'is', 'said', 'that', 'nobody', 'has', 'solved', 'the', 'problem', 'yet', '.', '</s>'], ['<s>', 'our', 'project', 'crashed', 'and', 'burned', '.', '</s>']]


In [33]:
def build_ngram(data, n):
    '''
    Parameters:
    data (list of lists): each list is a sentence of the text 
    n (int): size of the n-gram
    
    Returns:
    prob (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    '''
    total_number_words = 0
    counts = defaultdict(lambda: defaultdict(lambda: 0.0))

    for sentence in data:
        sentence = tuple(sentence)
        ## FILL CODE
        # dict can be indexed by tuples
        # store in the same dict all the ngrams
        # by using the context as a key and the word as a value

        for w in range(len( sentence)):
          total_number_words +=1
          for i in range(n):
            if w-i <0:
              break
            counts[sentence[w-i:w]][sentence [w]] +=1



    prob = defaultdict(lambda: defaultdict(lambda: 0.0))
    # Build the probabilities from the counts
    # Be careful with how you normalize!

    for context in counts.keys():

    ## FILL CODE
      den=0
      for i in counts[context].keys():
       den +=counts[context][i]

      for i in counts[context].keys():

        prob[context][i] =counts[context][i]/den

    return prob

In [34]:
# RUN TO BUILD NGRAM MODEL

n = 4
print("build ngram model with n = ", n)
model = build_ngram(train_data, n)

build ngram model with n =  4


Here, implement a recursive function over shorter and shorter context to compute a "stupid backoff model". An interpolation model can also be implemented this way.

In [242]:
# model['i', 'should']['answer']
# model['i', 'love']['you']

In [241]:
# 'you' in model['i', 'love'] 
# ('i', 'love') in model

In [37]:
def get_prob(model, context, w):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    context (list of strings): a sentence
    w(string): the word we need to find it's probability given the context
    
    Retunrs:
    prob(float): probability of this word given the context 
    '''

    # code a recursive function over 
    # smaller and smaller context
    # to compute the backoff model
    
    ## FILL CODE
    if context in model and w in model[context]:
      return model[context][w]

    else :
      return 0.4*get_prob(model, context[1:], w) # recursive


In [212]:
def perplexity(model, data, n):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    } 
    data (list of lists): each list is a sentence of the text
    n(int): size of the n-gram
    
    Retunrs:
    perp(float): the perplexity of the model 
    '''
    ## FILL CODE

    T,perplexity=0,0.0
    for sentence in data:
     sentence = tuple(sentence)
     for w in range(len(sentence)):
       T+=1
       c=min(n-1,w)
       context=sentence[w-c:w]
       perplexity+=-math.log(get_prob(model, context, sentence[w]))

    perplexity=perplexity/T
    



    return math.exp(perplexity)

In [213]:
# COMPUTE PERPLEXITY ON VALIDATION SET

print("The perplexity is", perplexity(model, valid_data, n=4))

The perplexity is 39.730815053864696


In [214]:
def get_proba_distrib(model, context):
    ## need to get the the words after the context and their probability of appearance
    ## after this context 
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    context (list of strings): the sentence we need to find the words after it and 
    thier probabilites
    
    Retunrs:
    words_and_probs(dic): {word: probability of word given context}
    
    '''
    # code a recursive function over context
    # to find the longest available ngram
    
    ## FILL CODE
    if context in model :
      return model[context]

    else :
      return get_proba_distrib(model, context[1:]) # recursive
    
    # return None

In [215]:
# get_proba_distrib(model, ('i','love'))


In [240]:
# get_proba_distrib(model, ('i','love'))['her']

In [219]:
def generate(model):
    '''
    Parameters: 
    model (dictionary of dictionary)
    {
        context: {word:probability of this word given context}
    }
    
    Returns:
    sentence (list of strings): a sentence sampled according to the language model. 
    

    '''
    # generate a sentence. A sentence starts with a <s> and ends with a </s>
    # Possiblly a use function is:
    # np.random.choice(x, 1, p = y)

    # where x is a list of things to sample from
    # and y is a list of probability (of the same length as x)
    sentence = ["<s>"]
    while sentence[-1] != "</s>" and len(sentence)<100:
        ## FILL CODE

        probs=get_proba_distrib(model,tuple(sentence))

        word=np.random.choice(list(probs.keys()), 1,list(probs.values()))

        sentence.append(word[0])

    return sentence

In [239]:
# GENERATE A SENTENCE FROM THE MODEL

print("Generated sentence: ",generate(model))

Generated sentence:  ['<s>', 'fear', 'nothing', '.', '</s>']


Once you are done implementing the model, evaluation and generation code, you can try changing the value of `n`, and play with a larger training set (`train2.txt` and `valid2.txt`). You can also try to implement an interpolation model.